In [ ]:
import pandas as pd

from model import get_model
from dataset import SpotDataset
from procedures import Evaluate
from utils import get_name, setup_logging

DIR = get_name()
print(DIR)

In [ ]:
setup_logging(log_level="INFO", log_file=DIR + "/evaluation.log")

# Load data

In [ ]:
train_df = pd.read_pickle(f"{DIR}/data/train_df.pkl")
val_df = pd.read_pickle(f"{DIR}/data/val_df.pkl")
test_df = pd.read_pickle(f"{DIR}/data/test_df.pkl")
instance_info_df = pd.read_pickle(f"{DIR}/data/instance_info_df.pkl")

train_df = pd.concat([train_df, val_df], ignore_index=True)

print(f"Number of different id_instances: {test_df['id_instance'].nunique()}")
test_df.info()

# Model Evaluation

In [ ]:
# Create test dataset before model initialization to derive feature sizes
test_dataset = SpotDataset(test_df, instance_info_df, DIR)

In [ ]:
# Pass test_dataset to get_model to automatically derive feature sizes
model = get_model(DIR, test_dataset)
model.load()

evaluator = Evaluate(model, test_dataset, DIR)

In [ ]:
metrics = evaluator.evaluate_all()